In [10]:
import hail as hl 
import subprocess
import os

### check if file paths are accessible 

In [ ]:
# if the path is accessible, return 'True' if not return 'False'
def path_works(path):
    if subprocess.call(['gsutil', 'ls', path]) == 0:
        return(True)
    else:
        return(False)

In [ ]:
paths =["gs://fc-1b3c48d4-1260-4477-ab89-9944a365a545/SCZ_Pulver_JHU_WES/C1622/Exome/P887751002D/v1/P887751002D.cram","gs://fc-6723cd5f-50f8-4828-a8e5-df42ad20af86/dalio_dutchwave2_Neale_Ophoff_bipolardisorder_exome/C2147/Exome/431-BG01077/v1/431-BG01077.cram","gs://fc-6723cd5f-50f8-4828-a8e5-df42ad20af86/dalio_dutchwave2_Neale_Ophoff_bipolardisorder_exome/C2147/Exome/431-BG01274/v1/431-BG01274.cram","gs://fc-6723cd5f-50f8-4828-a8e5-df42ad20af86/dalio_dutchwave2_Neale_Ophoff_bipolardisorder_exome/C2147/Exome/431-BG01320/v1/431-BG01320.cram"]

path_access = [] # for file paths that are accessible 
path_no_access = [] # for file paths that are NOT accessible 

# #with hl.hadoop_open('gs://imary116/data/cram_paths.txt') as paths:
for path in paths:
    if path_works(path):
        path_access.append(path)
    else:
        path_no_access.append(path)

### Part 1: by region

In [68]:
paths = ! gsutil ls gs://imary116/data/sampled100_per_pdo/output_files/coverage/region

In [69]:
# list of pdos IDs 
pdos = []
for p in paths:
    # only extract the pdo ID from the path - remove _region.ht/ from the end too
    pdo = os.path.splitext(os.path.basename(os.path.splitext(p[:-1])[0]))[0].split('_')[0]
    pdos.append(pdo)

In [71]:
#p = ["PDO-1294", "PDO-17148", "PDO-17294", "PDO-17755", "PDO-17756", "PDO-19811", "PDO-2980", "PDO-4831", "PDO-529", "PDO-6098", "PDO-6099", "PDO-6569", "PDO-6706", "PDO-6979", "PDO-74", "PDO-7756", "PDO-925"] 
# path = 'gs://imary116/data/sampled100_per_pdo/output_files/coverage/region/' # path to files 
# ext = '_region.ht' # file extension 


# full_pdo_paths = [path + s + ext for s in pdos] # make a complete path to pdo files 

In [81]:
# read all "region" pdo hail tables as one 
tables = [hl.read_table(t) for t in paths]
region_ht = hl.Table.union(*tables)

# sanity check 
region_ht.count() #1583374080 
# the above # is correct because we are looking at 1237011 regions (bed file) across all pdos (1280 samples in total) 
# 1237011*1280 = 1583374080

Initializing Hail with default parameters...
Running on Apache Spark version 3.1.1
SparkUI available at http://mty-m.c.daly-neale-sczmeta.internal:33591
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.65-367cf1874d85
LOGGING: writing to /home/hail/hail-20210510-1134-0.2.65-367cf1874d85.log
2021-05-10 11:34:24 Hail: WARN: Name collision: field 'sample' already in object dict. 
  This field must be referenced with __getitem__ syntax: obj['sample']


1583374080

In [82]:
region_ht.show(5)

,,,,
f0,f1,f2,f3,sample
str,int32,int32,float64,str
"""chr1""",11869,12227,8.00e-01,"""MH0126699"""
"""chr1""",12010,12057,2.80e-01,"""MH0126699"""
"""chr1""",12179,12227,1.00e+00,"""MH0126699"""
"""chr1""",12613,12721,6.10e+00,"""MH0126699"""
"""chr1""",12613,12697,6.46e+00,"""MH0126699"""


In [83]:
# import tsv file with pdo membership, cram paths and sample_ID for annotation purposes 
annot_table = hl.import_table('gs://imary116/data/sampled100_per_pdo/input_files/pdo17_crampath_sampleID.tsv', impute =True, delimiter='\t').key_by('sample_ID')

2021-05-10 11:38:51 Hail: INFO: Reading table to impute column types
2021-05-10 11:38:52 Hail: INFO: Finished type imputation
  Loading field 'pdo' as type str (imputed)
  Loading field 'cram' as type str (imputed)
  Loading field 'sample_ID' as type str (imputed)


In [84]:
annot_table.show(5)

,,
pdo,cram,sample_ID
str,str,str
"""PDO-6569""","""gs://fc-56e5af0e-4889-4845-a04e-1d07c7c60fb2/GPC_Pato_Latino_AA_WES/C1629/Exome/00C04941/v1/00C04941.cram""","""00C04941"""
"""PDO-6706""","""gs://fc-56e5af0e-4889-4845-a04e-1d07c7c60fb2/GPC_Pato_Latino_AA_WES/C1928/Exome/02C10755/v1/02C10755.cram""","""02C10755"""
"""PDO-6569""","""gs://fc-56e5af0e-4889-4845-a04e-1d07c7c60fb2/GPC_Pato_Latino_AA_WES/C1629/Exome/02C12137/v1/02C12137.cram""","""02C12137"""
"""PDO-6569""","""gs://fc-56e5af0e-4889-4845-a04e-1d07c7c60fb2/GPC_Pato_Latino_AA_WES/C1629/Exome/03C17467/v1/03C17467.cram""","""03C17467"""
"""PDO-6706""","""gs://fc-56e5af0e-4889-4845-a04e-1d07c7c60fb2/GPC_Pato_Latino_AA_WES/C1928/Exome/03C20282/v1/03C20282.cram""","""03C20282"""


In [85]:
region_ht = region_ht.annotate(pdo = annot_table[region_ht['sample']].pdo)

# sanity check 
region_ht.aggregate(hl.agg.collect_as_set(region_ht.pdo)) # all pdos are there 
#region_ht.aggregate(hl.agg.counter(region_ht.pdo)) # num of records for each pdo - should add up to 1583374080 (ht.count()) 

2021-05-10 11:39:51 Hail: INFO: Coerced sorted dataset
2021-05-10 11:39:51 Hail: INFO: Coerced dataset with out-of-order partitions.
2021-05-10 11:39:52 Hail: INFO: Ordering unsorted dataset with network shuffle


frozenset({'PDO-1294',
           'PDO-17148',
           'PDO-17294',
           'PDO-17755',
           'PDO-17756',
           'PDO-19811',
           'PDO-2980',
           'PDO-4831',
           'PDO-529',
           'PDO-6098',
           'PDO-6099',
           'PDO-6569',
           'PDO-6706',
           'PDO-6979',
           'PDO-74',
           'PDO-7756',
           'PDO-925'})

In [87]:
region_ht.show(5)

2021-05-10 11:42:37 Hail: INFO: Coerced sorted dataset
2021-05-10 11:42:37 Hail: INFO: Coerced dataset with out-of-order partitions.
2021-05-10 11:42:37 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,
f0,f1,f2,f3,sample,pdo
str,int32,int32,float64,str,str
"""chr1""",11869,12227,5.94e+01,"""00C04941""","""PDO-6569"""
"""chr1""",12010,12057,2.94e+01,"""00C04941""","""PDO-6569"""
"""chr1""",12179,12227,1.29e+02,"""00C04941""","""PDO-6569"""
"""chr1""",12613,12721,2.36e+02,"""00C04941""","""PDO-6569"""
"""chr1""",12613,12697,2.40e+02,"""00C04941""","""PDO-6569"""


In [88]:
region_ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'f0': str 
    'f1': int32 
    'f2': int32 
    'f3': float64 
    'sample': str 
    'pdo': str 
----------------------------------------
Key: []
----------------------------------------


In [89]:
# if the mean coverage (f3 column) >= 10, annotate it with a 1 and if it is not, annotate with a 0
region_ht = region_ht.annotate(mean_coverage10 = hl.if_else(region_ht.f3 >= 10,1,0))

In [90]:
region_ht.show(5)

2021-05-10 11:43:23 Hail: INFO: Coerced sorted dataset
2021-05-10 11:43:23 Hail: INFO: Coerced dataset with out-of-order partitions.
2021-05-10 11:43:24 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,,
f0,f1,f2,f3,sample,pdo,mean_coverage10
str,int32,int32,float64,str,str,int32
"""chr1""",11869,12227,5.94e+01,"""00C04941""","""PDO-6569""",1
"""chr1""",12010,12057,2.94e+01,"""00C04941""","""PDO-6569""",1
"""chr1""",12179,12227,1.29e+02,"""00C04941""","""PDO-6569""",1
"""chr1""",12613,12721,2.36e+02,"""00C04941""","""PDO-6569""",1
"""chr1""",12613,12697,2.40e+02,"""00C04941""","""PDO-6569""",1


In [104]:
# Do the following for each pdo (group by pdo first)
# For each region (chr:start:end), calculate the proportion of samples that have a mean coverage >=10 
# Calculate total number of samples in the same reagion that have mean coverage of >=10 (mean_cov10), total number of samples within that region (total), and the proportion which is mean_cov10 divided by the total (proportion)
prop_by_region = region_ht.group_by(region_ht.pdo,region_ht.f0,region_ht.f1,region_ht.f2).aggregate(
    mean_cov10 = hl.agg.count_where(region_ht.mean_coverage10 == 1),
    total = hl.agg.count(),
    proportion = hl.agg.fraction(region_ht.mean_coverage10 == 1))

In [105]:
prop_by_region.show(5)

2021-05-10 12:07:24 Hail: INFO: Coerced sorted dataset
2021-05-10 12:07:24 Hail: INFO: Coerced dataset with out-of-order partitions.
2021-05-10 12:07:24 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-10 12:16:43 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,,
pdo,f0,f1,f2,mean_cov10,total,proportion
str,str,int32,int32,int64,int64,float64
"""PDO-1294""","""chr1""",11869,12227,0,96,0.00e+00
"""PDO-1294""","""chr1""",12010,12057,0,96,0.00e+00
"""PDO-1294""","""chr1""",12179,12227,1,96,1.04e-02
"""PDO-1294""","""chr1""",12613,12697,32,96,3.33e-01
"""PDO-1294""","""chr1""",12613,12721,23,96,2.40e-01


In [110]:
prop_by_region.count()

2021-05-10 12:46:10 Hail: INFO: Coerced sorted dataset
2021-05-10 12:46:10 Hail: INFO: Coerced dataset with out-of-order partitions.
2021-05-10 12:46:10 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-10 12:59:08 Hail: INFO: Ordering unsorted dataset with network shuffle


9920639

In [107]:
pbr = region_ht.group_by(region_ht.f0,region_ht.f1,region_ht.f2).aggregate(mean_cov10 = hl.agg.count_where(region_ht.mean_coverage10 == 1),
                                                                           total = hl.agg.count(),
                                                                           proportion = hl.agg.fraction(region_ht.mean_coverage10 == 1))

pbr.show(5)

2021-05-10 12:28:08 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,
f0,f1,f2,mean_cov10,total,proportion
str,int32,int32,int64,int64,float64
"""chr1""",11869,12227,775,1280,6.05e-01
"""chr1""",12010,12057,772,1280,6.03e-01
"""chr1""",12179,12227,781,1280,6.10e-01
"""chr1""",12613,12697,864,1280,6.75e-01
"""chr1""",12613,12721,846,1280,6.61e-01


In [108]:
pbr.count()

2021-05-10 12:41:17 Hail: INFO: Ordering unsorted dataset with network shuffle


583567

In [118]:
# should be the same as pbr 
check = prop_by_region.group_by(prop_by_region.f0,prop_by_region.f1,prop_by_region.f2).aggregate(mean_cov10 = hl.agg.sum(prop_by_region.mean_cov10),
                                                                           total = hl.agg.sum(prop_by_region.total),
                                                                           proportion = hl.agg.sum(prop_by_region.proportion))
check.show(5)

2021-05-10 13:13:51 Hail: INFO: Coerced sorted dataset
2021-05-10 13:13:51 Hail: INFO: Coerced dataset with out-of-order partitions.
2021-05-10 13:13:52 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-10 13:25:20 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-10 13:28:12 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,
f0,f1,f2,mean_cov10,total,proportion
str,int32,int32,int64,int64,float64
"""chr1""",11869,12227,775,1280,7.99e+00
"""chr1""",12010,12057,772,1280,7.96e+00
"""chr1""",12179,12227,781,1280,8.07e+00
"""chr1""",12613,12697,864,1280,9.55e+00
"""chr1""",12613,12721,846,1280,9.25e+00


In [119]:
check.count()

2021-05-10 14:39:27 Hail: INFO: Coerced sorted dataset
2021-05-10 14:39:27 Hail: INFO: Coerced dataset with out-of-order partitions.
2021-05-10 14:39:28 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-10 14:49:58 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-10 14:52:41 Hail: INFO: Ordering unsorted dataset with network shuffle


583567

### Part 2: by sample

In [111]:
# calculate the length of each interval (f2 - f1) 
region_ht = region_ht.annotate(region_len = region_ht.f2 - region_ht.f1)

In [112]:
region_ht.show(5)

2021-05-10 13:02:37 Hail: INFO: Coerced sorted dataset
2021-05-10 13:02:37 Hail: INFO: Coerced dataset with out-of-order partitions.
2021-05-10 13:02:37 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,,,,
f0,f1,f2,f3,sample,pdo,mean_coverage10,region_len
str,int32,int32,float64,str,str,int32,int32
"""chr1""",11869,12227,5.94e+01,"""00C04941""","""PDO-6569""",1,358
"""chr1""",12010,12057,2.94e+01,"""00C04941""","""PDO-6569""",1,47
"""chr1""",12179,12227,1.29e+02,"""00C04941""","""PDO-6569""",1,48
"""chr1""",12613,12721,2.36e+02,"""00C04941""","""PDO-6569""",1,108
"""chr1""",12613,12697,2.40e+02,"""00C04941""","""PDO-6569""",1,84


In [113]:
# add up the interval values per sample (total)
# multiply the 1 v 0 column (mean_coverage10) with the length of the interval (region_len) and then add the values up (meancov10_sum) 
interval_sum_by_sample = region_ht.group_by('sample').aggregate(
    meancov10_sum = hl.agg.sum(region_ht.mean_coverage10 * region_ht.region_len),
    total = hl.agg.sum(region_ht.region_len))

# calculate the proportion 
interval_sum_by_sample = interval_sum_by_sample.annotate(prop = interval_sum_by_sample.meancov10_sum/interval_sum_by_sample.total)

In [114]:
interval_sum_by_sample.show(5)

2021-05-10 13:05:54 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,
sample,meancov10_sum,total,prop
str,int64,int64,float32
"""00C04941""",210003383,308140157,6.82e-01
"""02C10755""",212874629,308140157,6.91e-01
"""02C12137""",207937296,308140157,6.75e-01
"""03C17467""",211777007,308140157,6.87e-01
"""03C20282""",207828645,308140157,6.74e-01


In [115]:
# annotate the pdo over the interval_sum_by_sample table (one to one sample per pdo) 
interval_sum_by_sample = interval_sum_by_sample.annotate(pdo = annot_table[interval_sum_by_sample['sample']].pdo)

#pdo_interval_sum_by_sample = interval_sum_by_sample.annotate(**pdo_cram_annot[interval_sum_by_sample['sample']])

In [116]:
interval_sum_by_sample.show(5)

2021-05-10 13:09:45 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-10 13:09:47 Hail: INFO: Ordering unsorted dataset with network shuffle


,,,,
sample,meancov10_sum,total,prop,pdo
str,int64,int64,float32,str
"""00C04941""",210003383,308140157,6.82e-01,"""PDO-6569"""
"""02C10755""",212874629,308140157,6.91e-01,"""PDO-6706"""
"""02C12137""",207937296,308140157,6.75e-01,"""PDO-6569"""
"""03C17467""",211777007,308140157,6.87e-01,"""PDO-6569"""
"""03C20282""",207828645,308140157,6.74e-01,"""PDO-6706"""


In [117]:
# export above table as a tsv for plotting the samples as box plots grouped by pdo
# export is for tsv 
# write is for ht 
interval_sum_by_sample.export('gs://imary116/data/sampled100_per_pdo/output_files/interval_sum_by_sample.tsv')

2021-05-10 13:12:21 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-10 13:12:23 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-05-10 13:12:53 Hail: INFO: merging 1271 files totalling 62.4K...
2021-05-10 13:13:00 Hail: INFO: while writing:
    gs://imary116/data/sampled100_per_pdo/output_files/interval_sum_by_sample.tsv
  merge time: 6.944s


-

#### Sanity check 

In [34]:
interval_sum_by_sample.count()

2021-04-30 20:39:12 Hail: INFO: Ordering unsorted dataset with network shuffle


1271

In [41]:
len(region_ht.aggregate(hl.agg.collect_as_set(region_ht['sample'])))

1271